In [1]:
import pandas as pd
from collections import defaultdict
import dvc.api
from tqdm import tqdm
import numpy as np

In [2]:
df_original = pd.read_csv(dvc.api.get_url('modelling/notebooks/query_attr_extract_appen_label/appen_query_attribution_batch1.csv', 
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'))

In [3]:
# df = pd.read_excel('appen/output_batch/Query Attribution_02.03.23.xlsx')
df = pd.read_excel('appen/output_batch_correct/Query Attribution_02.03.23.xlsx')

In [4]:
df_original.head(2)

,query,sample_method,label_ordering,top_query_classification_taxonomy
0,dog cat eater,uniform,0,Home & Garden > Pet Products > Cat Supplies > ...
1,fish holder tool,head,18,Sports > Fishing > Fishing Tools


In [5]:
df_original['label_ordering'].isna().mean()

0.0

In [6]:
df = df.merge(df_original[['label_ordering', 'query', 'top_query_classification_taxonomy']].rename(columns={
    'query': 'query_original',
    'label_ordering': 'Label Ordering'
}), on='Label Ordering', how='inner')

In [7]:
(df['query_original'] != df['Query']).mean()

0.0646146459864236

In [8]:
df[df['query_original'] != df['Query']][['Label Ordering', 'Query', 'query_original', 'Rater Answer', 'Query Category']].sample(2).to_dict('records')

[{'Label Ordering': 1307978,
  'Query': 'lanÃ§ador de teia do homem-aranha que atira',
  'query_original': 'lançador de teia do homem-aranha que atira',
  'Rater Answer': nan,
  'Query Category': 'Home & Garden > Kitchen,Dining & Bar > Measuring Tools & Scales > Kitchen Scales'},
 {'Label Ordering': 715118,
  'Query': 'é\x9d´ ãƒ¡ãƒ³ã‚º ãƒŠã‚¤ã‚\xad',
  'query_original': '靴 メンズ ナイキ',
  'Rater Answer': nan,
  'Query Category': "Shoes > Men's Shoes > Men's Casual Shoes"}]

In [13]:
(df['Query Category'] != df['top_query_classification_taxonomy']).mean()

0.0

In [41]:
errors = []
correct = []
empty = []
for i in tqdm(df[['Query', 'query_original', 'Rater Answer', 'Query Category']].to_dict('records')):
    res = [] if ((not isinstance(i['Rater Answer'], str)) and (np.isnan(i['Rater Answer']))) else i['Rater Answer'].split('\n')
    if len(res) == 0:
        empty.append(i)
    is_correct = True
    for j in res:
        if not j.startswith(i['Query Category']):
            errors.append(i)
            is_correct = False
            break
    if is_correct:
        correct.append(i)
len(correct) / len(df), len(errors) / len(df), len(empty) / len(df)

100%|██████████| 74983/74983 [00:00<00:00, 453863.44it/s]


(0.7627995679020578, 0.2372004320979422, 0.1449128469119667)

In [42]:
correct[:5]

[{'Query': 'fish holder tool',
  'query_original': 'fish holder tool',
  'Rater Answer': 'Sports > Fishing > Fishing Tools > fish holder',
  'Query Category': 'Sports > Fishing > Fishing Tools'},
 {'Query': 'medayas de oro',
  'query_original': 'medayas de oro',
  'Rater Answer': 'Jewelry & Accessories > Fine Jewelry > Necklaces > Materials > Gold\nJewelry & Accessories > Fine Jewelry > Necklaces > Occasion & Lifestyle > Honor award',
  'Query Category': 'Jewelry & Accessories > Fine Jewelry > Necklaces'},
 {'Query': 'case para huawei y9 prime 2019',
  'query_original': 'case para huawei y9 prime 2019',
  'Rater Answer': 'Cellphones & Telecommunications > Mobile Phone Accessories > Phone Case & Covers > Brand > huawei y9 prime 2019',
  'Query Category': 'Cellphones & Telecommunications > Mobile Phone Accessories > Phone Case & Covers'},
 {'Query': 'lace dots meter',
  'query_original': 'lace dots meter',
  'Rater Answer': 'Home & Garden > Arts, Crafts & Sewing > Apparel Sewing & Fabric

In [43]:
# query errors
errors[:5]

[{'Query': 'molde gesso 3d silicone',
  'query_original': 'molde gesso 3d silicone',
  'Rater Answer': 'Home & Garden > Arts, Crafts & Sewing > Pottery & Ceramics > Clay Molds > Materials > Silicone\nHome & Garden > Arts, Crafts & Sewing > Pottery & Ceramics > Clay Molds > Pattern > 3D',
  'Query Category': 'Home & Garden > Arts, Crafts & Sewing > Button & Badge Making > Maker Molds'},
 {'Query': 'gucci pouch mens',
  'query_original': 'gucci pouch mens',
  'Rater Answer': "Luggage & Bags > Women's Bags > Waist Packs > Department > Men > gucci",
  'Query Category': "Luggage & Bags > Men's Bags > Waist Packs"},
 {'Query': 'housse de couette 90x190',
  'query_original': 'housse de couette 90x190',
  'Rater Answer': 'Home & Garden > Home Textile > Duvet Cover Sets > Size > 90x190',
  'Query Category': 'Home & Garden > Home Textile > Bedding > Duvet Covers'},
 {'Query': 'caixa porta joias',
  'query_original': 'caixa porta joias',
  'Rater Answer': 'Home & Garden > Home Storage & Organizat